In [44]:
import csv
import pandas as pd
import plotly.express as px
import numpy as np

# Read and skip 17 lines
csv_file_path = 'resource_per_entity.csv'
data = []

allow_list = [
    "scrambler_mm",
    "rom_scrambler_config",
    "rom_scrambler_config_reader",
    "scrambler",
    "drbg_consumer",
    "drbg_synchronizer",
    "hash_drbg_sha256",
    "hash_drbg",
    "sha256_core",
    "line_rotator",
    "sequence_detector",
    "sequence_generator",
    "sequence_generator_switch",
    "sync_parser",
    "sld_hub",
]
allow_list = set(allow_list)

not_end_nodes = ["scrambler_mm", "scrambler", "hash_drbg_sha256"]
not_end_nodes = set(not_end_nodes)

with open(csv_file_path, 'r') as f:
    for _ in range(18):
        next(f)
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

# Prepare data
treemap_rows = []
stack = []
        
for row in data:
    try:
        # if "alt" in row[0]: #skip altera components
        #     continue
        hierarchy = row[0].split("|")

        
        depth = len(hierarchy[0]) // 3
        while len(stack) > depth:
            stack.pop()
            
        stack.append(hierarchy[1].split(":")[0])
        if stack[-1] not in not_end_nodes:
            aluts = int(row[1].split('(')[0])
            block_memory_bits = int(row[3])
        else:
            aluts = int(row[1].split("(")[1][:-1])
            block_memory_bits = 1
    except (ValueError, IndexError):
        continue
    if stack[-1] not in allow_list:
        continue
    
        
    
    if aluts > 0:  # <---- ONLY include if ALUTs > 0 (prevent zero-weight crash)
        
        if (stack[-1] in ["hash_drbg_sha256", "scrambler_mm", "sld_hub"]):
            name = " ".join(stack[-1].split("_"))
        else:
            name = "<br>".join(stack[-1].split("_"))

        if depth != 0:
            if (stack[-2] in ["hash_drbg_sha256", "scrambler_mm", "sld_hub"]):
                parent= " ".join(stack[-2].split("_"))
            else:
                parent= "<br>".join(stack[-2].split("_"))
        else:
            parent = ""
        treemap_rows.append({
            "name": name,
            "parent": parent,
            "ALUTs": aluts,
            "BlockMemoryBits": block_memory_bits
        })

# Convert to DataFrame
df = pd.DataFrame(treemap_rows)
# df.loc[df.name == "scrambler mm", "ALUTs"] -= list(df.loc[df.name == "sld hub", "ALUTs"])[0]
# df.loc[df.name == "scrambler mm", "BlockMemoryBits"] -= list(df.loc[df.name == "sld hub", "BlockMemoryBits"])[0]
df.drop(df.loc[df.name == "sld hub"].index, inplace=True)

# # Plot treemap
MIN_ALUT_THRESHOLD = 512  # Set a minimum display size (in ALUTs)
df.set_index("name")
# df['ALUTs'] = df['ALUTs'] - df[df.name == "sld hub"]["ALUTs"]

df['ALUTs_adj'] = df['ALUTs'].apply(lambda x: x if x >= MIN_ALUT_THRESHOLD else MIN_ALUT_THRESHOLD)
df['ALUTs_log2'] = np.log2(df['ALUTs'])
# np.log
# df['ALUTs_log2'] = df['ALUTs'] ** 0.5
df["BlockMemoryBits_log2"] = np.log2(df['BlockMemoryBits'])
# df["BlockMemoryBits_log2"] = df['BlockMemoryBits'] ** 0.1
fig = px.treemap(
    df,
    names="name",
    parents="parent",
    values="ALUTs_log2",
    color="ALUTs_log2",  # You can change to "BlockMemoryBits" if preferred
    color_continuous_scale="Rainbow",
    title="ALUTs Utilization",
)

max_log2 = np.ceil(df["ALUTs_log2"].max()).astype(int) + 1  # safest casting
tickvals = list(range(max_log2 + 1))  # log2 scale values
ticktext = [str(2**i) for i in tickvals]  # actual ALUTs (non-log scale)

fig.update_layout(
    width=1000,                # Wider figure
    height=600,                # Taller figure
    margin=dict(t=100, l=25, r=25, b=25),
    font=dict(size=18),         # Larger font size for all text,
    uniformtext=dict(minsize=18),
    coloraxis=dict(
        # color=df['ALUTs_log2'],
        showscale=True,
        colorscale='Rainbow',
        # dtick="log",
        cmin = 0,
        cmax = int(tickvals[-1]),
        colorbar=dict(
            title='ALUTs',
            tickvals=tickvals,
            ticktext=ticktext,
        ),
    )
)
fig.show()

max_log2 = np.ceil(df["BlockMemoryBits_log2"].max()).astype(int)  # safest casting
tickvals = list(range(max_log2 + 1))  # log2 scale values
ticktext = [str(2**i) for i in tickvals]  # actual ALUTs (non-log scale)

fig = px.treemap(
    df,
    names="name",
    parents="parent",
    values="BlockMemoryBits_log2",
    color="BlockMemoryBits_log2",  # You can change to "BlockMemoryBits" if preferred
    color_continuous_scale="Rainbow",
    title="Memory Utilization"
)

fig.update_layout(
    width=1000,                # Wider figure
    height=600,                # Taller figure
    margin=dict(t=100, l=25, r=25, b=25),
    font=dict(size=18),         # Larger font size for all text,
    uniformtext=dict(minsize=18),
    coloraxis=dict(
        # color=df['ALUTs_log2'],
        showscale=True,
        colorscale='Turbo',
        # dtick="log",
        cmin = 0,
        cmax = int(tickvals[-1]),
        colorbar=dict(
            title='Memory Bits',
            tickvals=tickvals,
            ticktext=ticktext,
        ),
    )
)
# fig.show()


C:\Users\ADMIN\anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning:

divide by zero encountered in log2



34

In [161]:
"123 (323)".split("(")[1][:-1]

'323'

In [147]:
df.loc[df.name == "scrambler mm", "ALUTs"]

0    2629
Name: ALUTs, dtype: int64

In [80]:
print(*[f'"{i}"' for i in df.name], sep=",\n")

"scrambler_mm",
"rom_scrambler_config",
"altsyncram",
"altsyncram_72j1",
"sld_mod_ram_rom",
"sld_rom_sr",
"rom_scrambler_config_reader",
"scrambler",
"drbg_consumer",
"drbg_synchronizer",
"hash_drbg_sha256",
"hash_drbg",
"posedge_to_pulse",
"posedge_to_pulse",
"sha256_core",
"sha256_k_constants",
"sha256_w_mem",
"line_rotator",
"cut_position_interpolator",
"sequence_detector",
"sequence_generator",
"clk_div36",
"sequence_shiftreg_out",
"lpm_shiftreg",
"sequence_generator_switch",
"sync_parser",
"sld_hub",
"alt_sld_fab_with_jtag_input",
"alt_sld_fab",
"alt_sld_fab_alt_sld_fab",
"alt_sld_fab_alt_sld_fab_sldfabric",
"sld_jtag_hub",
"sld_rom_sr",
"sld_shadow_jsm"


In [ ]:
[
"scrambler_mm",
"rom_scrambler_config",
"rom_scrambler_config_reader",
"scrambler",
"drbg_consumer",
"drbg_synchronizer",
"hash_drbg_sha256",
"hash_drbg",
"sha256_core",
"line_rotator",
"sequence_detector",
"sequence_generator",
"sequence_generator_switch",
"sync_parser",
"sld_hub",
    ]